<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:200%;
           font-family:Arial;letter-spacing:0.5px">

<p width = 20%, style="padding: 10px;
              color:white;">
Model Tuning
              
</p>
</div>

Data Science Cohort Live NYC July 2022
<p>Phase 3: Topic 30</p>
<br>
<br>

<div align = "right">
<img src="Images/flatiron-school-logo.png" align = "right" width="200"/>
</div>
    
    

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats as stats

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import precision_score, recall_score, plot_confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV,\
cross_val_score, RandomizedSearchCV

from sklearn.preprocessing import OneHotEncoder, StandardScaler, Normalizer
from sklearn.impute import SimpleImputer

from sklearn.compose import ColumnTransformer

#### Hyperparameters
- Free knobs of a model used to **control the training process**.

<img src = "Images\levers.jpg" width = 350/><center> Tuning your model!! </center>

**Hyperparameters concerning the structure of cost function and model**

#### Hyperparameters
- Free levers of a model used to **control the training process**.
- $\lambda$: the regularization parameter for $|\textbf{w}|^2$ penalty.


#### Parameters

- **Found by training the model**
- Weights ($\textbf{w}$, etc.): 
    Logistic Regression, Linear Regression


#### Hyperparameters
- depth of decision tree
- gini impurity or entropy?

#### Parameters

- Locations in feature space where splits were made

**Hyperparameters concerning the optimization process**

- Can help get to solutions faster
- Get *better* convergence to cost function minimum

**Solvers for Logistic regression**: variants of gradient descent.

- lbfgs (variant of Newton method)
    - accounts for concavity of cost function
- sag (Stochastic Gradient Descent)
    - fast convergence
    - excellent for larger number of samples
- liblinear (Coordinate Descent)
    - better for larger number of features



Depending on the sort of problem and data:
- experiment with different values of these hyperparameters
- improve model performance.

Running through the entire process of train/test splitting and cross-validation yet again:

Load iris data

In [ ]:
iris_df = pd.read_csv('Data/Iris.csv').drop(columns = ['Id'])
iris_df.head()

Label encode the target class

In [ ]:
from sklearn.preprocessing import LabelEncoder
target_transform = LabelEncoder()
iris_df['Species'] = target_transform.fit_transform(iris_df['Species'])
iris_df.head()

Select two of the features and the target. Features are not obviously separable.
- Tuning will be important.

In [ ]:
col_list = ['SepalWidthCm', 'SepalLengthCm']
X = iris_df[col_list]
y = iris_df['Species']

In [ ]:
sns.pairplot(hue = 'Species', 
             vars = col_list,
             data = iris_df)
plt.show()

Now, before doing anything: Train/test split!

In [ ]:
# shuffle and split, stratify keeps target distribution same in train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size = 0.15, random_state = 42)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report

In [ ]:
def cross_validation(X_train, y_train, k, num_split = 10):
    
    X_train = X_train.values
    y_train = y_train.values
    
    score_train_list = []
    score_val_list = []
    
    for train_index, valid_index in KFold(n_splits = num_split).split(X_train):
        # train and validation splitting 
        X_train_fold, X_val_fold = X_train[train_index], X_train[valid_index]
        y_train_fold, y_val_fold = y_train[train_index], y_train[valid_index]

        #create/fit the Standard scaler on the train fold
        scaler = StandardScaler()
        X_tf_sc = scaler.fit_transform(X_train_fold)
        # transform validation fold
        X_vld_sc = scaler.transform(X_val_fold)

        # create/fit knearest neighbor
        knn = KNeighborsClassifier(n_neighbors = k)
        knn.fit(X_tf_sc, y_train_fold)
        
        # now how did we do?
        accuracy_train = knn.score(X_tf_sc, y_train_fold)
        accuracy_val = knn.score(X_vld_sc, y_val_fold)
        score_val_list.append(accuracy_val)
        score_train_list.append(accuracy_train)
    
    return {'k': k, 'train': np.mean(score_train_list), 'validation': np.mean(score_val_list)}

Now for tuning parameters need to loop:

In [ ]:
# empty dataframe
crossval_df = pd.DataFrame(columns = ['k', 'train', 'validation'])
# append results for each value of k
for k in np.arange(1,100):
    crossval_df = crossval_df.append(cross_validation(X_train, y_train, k, 5), ignore_index = True)


In [ ]:
crossval_df.head()

Technically, we did this correctly. But...

#### Whats the problem with this?

- Need to write unecessary code. Hard to read.
- Easy to mess up: track each data transformation carefully.
- Changing/adding transformations: possibly a pain.
- For loops for tuning, etc.

#### The goal:

- want to get best model in streamlined process.
- want to be able to plug in and out different transformations easily and transparently. 
- try multiple transformations/models at once
- want good reporting/ranking of different models.

####  First part of solution: The pipeline 

- Transformer fitting and model fitting
- Happen one right after the other.
- **Ideal: unify into a single process.**

Without the pipeline:
- have to repeat fit and transformer/predicting for every transformer and model

In [ ]:
# Define transformers (will adjust/massage the data)
imputer = SimpleImputer(strategy="median") # replaces missing values
std_scaler = StandardScaler() # scales the data

# Define the classifier (predictor) to train
rf_clf = DecisionTreeClassifier(random_state=42)

# Have the classifer (and full pipeline) learn/train/fit from the data
X_train_filled = imputer.fit_transform(X_train)
X_train_scaled = std_scaler.fit_transform(X_train_filled)
rf_clf.fit(X_train_scaled, y_train)

# Predict using the trained classifier (still need to do the transformations)
X_test_filled = imputer.transform(X_test)
X_test_scaled = std_scaler.transform(X_test_filled)
y_pred = rf_clf.predict(X_test_scaled)
print(y_pred)

Instead let's use the Pipeline object

In [ ]:
from sklearn.pipeline import Pipeline

# this will allow us to visualize the pipeline (may not be available in learn-env)
from sklearn import set_config
set_config(display= 'diagram')

Pipeline() constructor:
- takes in list of tuples (steps):
- user label of estimator/transformer in pipeline (a string)
- the transformer/estimator


Pipeline() object is a new estimator:
- does all steps in a defined order
- transformers in the beginning
- predictive model must be last step

Combining into a unified estimator-transformer:
- single .fit()
- single .predict()

**Does everything in one shot**

Let's check this out on fitting/predicting with simple train/test split:

In [ ]:
steps = [('imputer', SimpleImputer(strategy="median")), ('std_scaler', StandardScaler()),
        ('rf_clf', DecisionTreeClassifier(random_state=42))]

pipeline = Pipeline(steps)


# Train the pipeline (tranformations & predictor)
pipeline.fit(X_train, y_train)

# Predict using the pipeline (includes the transfomers & trained predictor)
predicted = pipeline.predict(X_test)
print(predicted)

Side by side (they do the same thing)

In [ ]:
# Define transformers (will adjust/massage the data)
imputer = SimpleImputer(strategy="median") # replaces missing values
std_scaler = StandardScaler() # scales the data

# Define the classifier (predictor) to train
rf_clf = DecisionTreeClassifier(random_state=42)

# Have the classifer (and full pipeline) learn/train/fit from the data
X_train_filled = imputer.fit_transform(X_train)
X_train_scaled = std_scaler.fit_transform(X_train_filled)
rf_clf.fit(X_train_scaled, y_train)

# Predict using the trained classifier (still need to do the transformations)
X_test_filled = imputer.transform(X_test)
X_test_scaled = std_scaler.transform(X_test_filled)
y_pred = rf_clf.predict(X_test_scaled)
print(y_pred)

In [ ]:
steps = [('imputer', SimpleImputer(strategy="median")), ('std_scaler', StandardScaler()),
        ('rf_clf', DecisionTreeClassifier(random_state=42))]

pipeline = Pipeline(steps)


# Train the pipeline (tranformations & predictor)
pipeline.fit(X_train, y_train)

# Predict using the pipeline (includes the transfomers & trained predictor)
predicted = pipeline.predict(X_test)
print(predicted)

Now lets do **manual** cross validation **with pipeline**

First advantage: can define the entire set of transformations outside cross-validation function:

In [ ]:
# Can define outside the cross validation
steps = [('imputer', SimpleImputer(strategy="median")), ('std_scaler', StandardScaler()),
        ('rf_clf', DecisionTreeClassifier(random_state=42))]

model_pipe = Pipeline(steps)
model_pipe

Second advantage: no keeping track of multiple variables.
- single .fit() and .predict()

In [ ]:
def cross_validation_pip(X_train, y_train, estimator, num_split = 10):
    
    X_train = X_train.values
    y_train = y_train.values
    
    score_train_list = []
    score_val_list = []
    
    for train_index, valid_index in KFold(n_splits = num_split).split(X_train):
        
        # train and validation splitting 
        X_train_fold, X_val_fold = X_train[train_index], X_train[valid_index]
        y_train_fold, y_val_fold = y_train[train_index], y_train[valid_index]

        estimator.fit(X_train_fold, y_train_fold)
        
        # now how did we do?
        accuracy_train = estimator.score(X_train_fold, y_train_fold)
        accuracy_val = estimator.score(X_val_fold, y_val_fold)
        score_val_list.append(accuracy_val)
        score_train_list.append(accuracy_train)
    
    return {'train': np.mean(score_train_list), 'validation': np.mean(score_val_list)}

In [ ]:
cross_validation_pip(X_train, y_train, pipeline)

Compare manual cross-validation **without** pipeline:

In [ ]:
def cross_validation(X_train, y_train, estimator, num_split = 10):
    
    X_train = X_train.values
    y_train = y_train.values
    
    score_train_list = []
    score_val_list = []
    
    for train_index, valid_index in KFold(n_splits = num_split).split(X_train):


        # train and validation splitting 
        X_train_fold, X_val_fold = X_train[train_index], X_train[valid_index]
        y_train_fold, y_val_fold = y_train[train_index], y_train[valid_index]
        
        # impute NaNs with medians
        
        simple_imp = SimpleImputer(strategy="median")
        X_tf_imp = simple_imp.fit_transform(X_train_fold)
        X_vld_imp = simple_imp.transform(X_val_fold)

        #create/fit the Standard scaler on the train fold
        scaler = StandardScaler()
        X_tf_sc = scaler.fit_transform(X_tf_imp)
        
        # transform validation fold
        X_vld_sc = scaler.transform(X_vld_imp)

        estimator.fit(X_tf_sc, y_train_fold)
        
        # now how did we do?
        accuracy_train = estimator.score(X_tf_sc, y_train_fold)
        accuracy_val = estimator.score(X_vld_sc, y_val_fold)
        score_val_list.append(accuracy_val)
        score_train_list.append(accuracy_train)
    
    return {'train': np.mean(score_train_list), 'validation': np.mean(score_val_list)}

In [ ]:
dec_tree = DecisionTreeClassifier(random_state=42)
cross_validation(X_train, y_train, dec_tree)

Same result as pipeline but:
- Much less customizable/plug and play
- Have to be careful and keep track of variables.
- Changing/adding steps requires changing code **carefully** in function
- Track changes and follow order of transformations:
    - train set
    - test set

<img src = "Images/pipeline_vs_manual.png" width = 600  /> <center>Pipeline: customizable box. <i>Takes care of the details.</i></center>

#### Swapping in Different Models & Transformers

Pipeline object has the steps as an attribute:
- Python list
- can modify pipeline progression by modifying attribute.

In [ ]:
model_pipe.steps

Change model to KNN:

In [ ]:
# pop the decision tree
model_pipe.steps.pop(-1)
# insert knn
model_pipe.steps.append(
['knn', KNeighborsClassifier()])

In [ ]:
model_pipe

Can change transformers in the middle:

In [ ]:
model_pipe.steps[1] = ('norm', Normalizer())
model_pipe

#### Pipelines
- Easily plug/play different transformers and models into our workflow

#### Pipelines clearly awesome, but:

Transformer/model pipeline hyperparameter tuning with cross-validation still pretty manual:

Want:
- Efficient hyperparameter tuning with minimal code
- Leakage prevention
- Automatic best model selection and reporting on the CV for each pipeline tried. 


#### The CV Grid Search

Scikit-Learn has a [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) class whose `fit()` method:

- takes in a grid of hyperparameters
- loops through all of them **in order**
- runs cross-validation properly scoring by chosen metric


- Returns all the models fitted
- Can easily access best model and its score

<center><img src = "Images/hyperparameter_tuning_cv_grid.webp" width = 400/></center>

#### GridSearchCV

In [ ]:
# let's define a new pipeline object 

steps = [('imputer', SimpleImputer(strategy="median")), ('std_scaler', StandardScaler()),
        ('knn', KNeighborsClassifier())]

model_pipe = Pipeline(steps)

CV takes in a grid dictionary:
- with pipeline: key must follow a specific format
- "pipelinename__hyperparameter"

With pipeline:
- Can change hyperparameters of model
- **Also** can change hyperparameters of transformers.

In [ ]:

pipe_grid = {'imputer__strategy': ["mean", "median"],
             'knn__n_neighbors': [3, 5, 7, 9, 11, 13, 15, 18],
             'knn__p': [1, 2, 3, 4]}

# note: default scoring is aaccuracy
gs_pipe = GridSearchCV(estimator=model_pipe, 
                       param_grid=pipe_grid)


In [ ]:
gs_pipe.fit(X_train, y_train);

In [ ]:
gs_pipe.best_params_

In [ ]:
gs_pipe.best_score_

Get the best estimator

In [ ]:
best_model = gs_pipe.best_estimator_
best_model

Evaluate on the test

In [ ]:
y_pred = best_model.predict(X_test)

In [ ]:
plot_confusion_matrix(best_model, X_test, y_test)

In [ ]:
print(classification_report(y_test, y_pred))

Can also get full report of cross validation with the pipeline estimators:
- e.g. a dataframe with all the models

In [ ]:
cv_result_df = pd.DataFrame(gs_pipe.cv_results_)
cv_result_df = cv_result_df.sort_values(by = 'rank_test_score')
cv_result_df.head()

Trying different metrics:
- Default scoring was accuracy
- But can try other with scoring = ''

- Takes string:
    - 'average_precision'
    - 'balanced_accuracy' (class averaged recall)
    - 'roc_auc', etc.

In [ ]:
pipe_grid = {'imputer__strategy': ["mean", "median"],
             'knn__n_neighbors': [3, 5, 7, 9, 11, 13, 15, 18],
             'knn__p': [1, 2, 3, 4]}

gs_pipe = GridSearchCV(estimator=model_pipe, 
                       param_grid=pipe_grid, scoring = 'precision_micro')

In [ ]:
gs_pipe.fit(X_train, y_train);
best_est = gs_pipe.best_estimator_

In [ ]:
best_est

Obviously not enough search parameters: still gets same model
- Also probably limited by intrinsic overlap in the data.
- Will see another example of trying different metrics.

#### In this case:
- Not many NaNs
- Selected subset of data didn't require multiple types of transformers
    - e.g. categorical vs. numeric columns
    - ordinal vs. nominal categoricals
    - feature engineering on column subsets (Polynomial, etc.)


#### Column Transformers

First step in:
- dealing with systematically imputing data in pipelines
- transforming features of various types in different ways

In [ ]:
salary_df = pd.read_csv('Data/adult.csv', usecols = ['age', 'occupation', 'education', 'native.country', 
                                                     'hours.per.week', 'sex', 'income'])
salary_df = salary_df[salary_df["native.country"] == "United-States"]
ed_list = ['10th','11th','HS-grad', 'Some-college', 'Assoc-acdm', 'Assoc-voc', 'Bachelors', 'Masters', 'Doctorate']
salary_df = salary_df[salary_df['education'].isin(ed_list)].drop(columns = ['native.country'])

Salary prediction problem:
- Predict whether someone makes >50K anually
- Numeric and categorical features require different preparation
- NaN imputation strategies specific to columns

Let's learn how to implement all this in the Pipeline

In [ ]:
salary_df.head()

In [ ]:
salary_df.info()

Nulls clearly encoded as '?'
- Would do EDA before figuring out how to clean/preprocess columns
- But lets go ahead and develop pipeline!

**Imputation**
- Numeric: impute with mean
- Education/occupation/sex: most common

**Encoding**
- Education: ordinal encoding
- Occupation/Sex: one-hot encoding

Then standardize and model.

In [ ]:
salary_df['education'].value_counts()

No nans? Guess again!

In [ ]:
salary_df.head()

In [ ]:
# replace with NaNs
salary_df = salary_df.replace({'?':np.nan})
salary_df.head(3)

In [ ]:
salary_df.info()

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
X_income = salary_df.drop(columns = ["income"])
y_income = salary_df['income']

In [ ]:
lab_enc = LabelEncoder()
y_income = lab_enc.fit_transform(y_income)

#### Before doing anything else

In [ ]:
X_inc_train, X_inc_test, y_inc_train, y_inc_test = train_test_split(X_income, y_income, test_size = 0.15)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.compose import ColumnTransformer

Create three separate transformation pipelines:
- Ordinal Categorical
- Nominal Categorical
- Numeric 

In [ ]:
X_income.head()

What do we want to use for each column?

## First define the separate pipelines

In [ ]:
numeric_pipeline = Pipeline([('numimputer', SimpleImputer(strategy = 'mean')), ('numnorm', StandardScaler())])

ordinal_pipeline = Pipeline([
    ('ordimputer', SimpleImputer(strategy = 'most_frequent')),
    ('ordenc', OrdinalEncoder(categories = [ed_list])),
    ('ordnorm', StandardScaler())])

nominal_pipeline = Pipeline([
    ('onehotimputer', SimpleImputer(strategy = 'most_frequent')),
    ('onehotenc', OneHotEncoder(sparse = False, drop = 'first')), 
    ('onehotnorm', MaxAbsScaler())])

ed_list

Now unite them with ColumnTransformer:
- specify columns each pipeline acts on.

In [ ]:
num_cols = X_inc_train.select_dtypes(['int', 'float']).columns

ct = ColumnTransformer(
    [("ordinalpipe", ordinal_pipeline, ['education']), 
     ("nominalpipe", nominal_pipeline, ['occupation','sex']),
     ("numpipe", numeric_pipeline, num_cols)])

num_cols

Viewing the transformed data:
- In order specified by column transformer

In [ ]:
pd.DataFrame(ct.fit_transform(X_inc_train)).head()

Let's view what our preprocessing pipeline does:

In [ ]:
ct

Takes specified columns:
- Applies ordinal pipeline to first set
- Nominal pipeline to second subset
- Numeric pipeline to last set

Joins result *in column transform order* at the end.

Column transformer: access/change individual steps in branches
- .named_transformers_ attribute as dictionary of transformers

In [ ]:
ct.named_transformers_

In [ ]:
ct.named_transformers_['ordinalpipe']

In [ ]:
ct.named_transformers_['nominalpipe'].steps.pop(-1)
ct.named_transformers_['nominalpipe'].steps.append([
    ('onehotnorm', StandardScaler())
])

In [ ]:
ct.named_transformers_

Swapped out MaxAbScaler with StandardScaler in nominalpipe:

At this point: 
- append modeling step
- build final pipeline: transformers + model
- tuck into gridCV for tuning.

In [ ]:
final_pipe = Pipeline([('preprocess', ct),
                      ('model',
                       KNeighborsClassifier())])
final_pipe

Fit the full transformer and model pipeline.

In [ ]:
final_pipe.fit(X_inc_train, y_inc_train)

In [ ]:
y_pred = final_pipe.predict(X_inc_test)

In [ ]:
plot_confusion_matrix(final_pipe, X_inc_test,
                      y_inc_test)

Probably needs some tuning. Let's look at the steps of final_pipe.

In [ ]:
print(final_pipe.steps)

In [ ]:
# going down each level in estimator with double underscore (level1__level2__estimator__argument)
final_pipe_grid = {'preprocess__numpipe__numimputer__strategy': ["mean", "median"],
             'model__n_neighbors': [3, 11, 15],
             'model__p': [1, 2]}

gs_full_pipe = GridSearchCV(estimator=final_pipe, 
                       param_grid=final_pipe_grid)

In [ ]:
gs_full_pipe.fit(X_inc_train,y_inc_train)

In [ ]:
gs_full_pipe.best_score_

In [ ]:
gs_full_pipe.best_params_

In [ ]:
best_model = gs_full_pipe.best_estimator_
y_pred = best_model.predict(X_inc_test)
plot_confusion_matrix(best_model,
                      X_inc_test,
                      y_inc_test)

Try a different scoring metric due to class imbalance

In [ ]:
# going down each level in estimator with double underscore (level1__level2__estimator__argument)
final_pipe_grid = {'preprocess__numpipe__numimputer__strategy': ["mean", "median"],
             'model__n_neighbors': [3, 11, 15],
             'model__p': [1, 2]}

gs_full_pipe2 = GridSearchCV(estimator=final_pipe, 
                       param_grid=final_pipe_grid, scoring = 'roc_auc')

In [ ]:
gs_full_pipe2.fit(X_inc_train, y_inc_train)
best_model2 = gs_full_pipe2.best_estimator_
plot_confusion_matrix(best_model2,
                      X_inc_test,
                      y_inc_test)

Scoring made a difference, but not that much. In some case will make a *huge* difference.

Probably would do much better with a better classifier and a better tuning range:
- But now you see how to address hyperparameters inside column transformers
- At any level of the full pipeline
- Change metrics you are tuning on

#### Feature Union

Given a subset of features:
- Want to transform feature subset in multiple ways (in parallel)
- Keep all of these transformed subsets as features
- **Feature Union** of multiple transformers

- E.g. PolynomialFeatures
- Function transformer (transform features according to function)

In [ ]:
from scipy.stats import norm
X = np.linspace(-1.7*np.pi, 1.5*np.pi, 100)
np.random.shuffle(X)
epsilon = norm.rvs(loc = 0, scale = 3.5, size = 100)
y = 5*np.sin(2*X) - 3*np.cos(X) + 2*X + .3*X**2 + epsilon
data_df = pd.DataFrame()
data_df['X'] = X
data_df['y'] = y

In [ ]:
data_df

In [ ]:
sns.scatterplot(x = 'X', y = 'y', data = 
                data_df)
plt.show()

In [ ]:
X_reshape = data_df['X'].values.reshape(-1,1)
y = data_df['y']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_reshape, y, test_size = 0.2)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import FeatureUnion
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error

Let's also define two custom functions:

In [ ]:
def sinefunc(x, c = 1):
    return np.sin(c*x)

In [ ]:
def cosfunc(x, d = 1):
    return np.cos(d*x)

In [ ]:
transformer_list = [('poly', PolynomialFeatures(include_bias = False)), 
                     ('sine', FunctionTransformer(sinefunc)), 
                     ('cosine', FunctionTransformer(cosfunc))
                   ]

# takes in input features, applies transformers in parallel to inputs, concatenates:
feat_union = FeatureUnion(transformer_list) 

In [ ]:
steps = [('feateng', feat_union), ('scaler', StandardScaler())]
preprocess_pipe = Pipeline(steps)
preprocess_pipe

In [ ]:
X_proc = preprocess_pipe.fit_transform(X_train)
X_proc[0:5]

First column is linear in X, second quadratic in X, then the sin and cosine terms.

- Check if all standard scaled:

In [ ]:
print(X_proc.mean(axis = 0))

In [ ]:
print(X_proc.std(axis = 0))

Finish pipeline and hyperparameter tune:

In [ ]:
steps = [('preprocess', preprocess_pipe), ('ridge', Ridge())]
regress_pipe = Pipeline(steps)
params = {'preprocess__feateng__poly__degree': [1, 2, 3, 4], 
          'preprocess__feateng__sine__kw_args': [{'c': 1},{'c': 2},{'c': 3}],
          'ridge__alpha': [1e-3, .01, .1, 1, 10, 100]}
cv = GridSearchCV(estimator = regress_pipe, param_grid = params, cv = 6)
cv.fit(X_train, y_train)
best_model = cv.best_estimator_

In [ ]:
best_model.steps

In [ ]:
cv.best_score_

In [ ]:
cv.best_params_

In [ ]:
pd.DataFrame(cv.cv_results_)

In [ ]:
best_model.fit(X_train, y_train)
y_predictions = best_model.predict(data_df['X'].values.reshape(-1,1))

In [ ]:
fig, ax = plt.subplots()
ax.scatter(X, y_predictions, label = 'fit')
ax.scatter(X, y, label = 'data')
ax.legend()
plt.show()

In [ ]:
y_pred = best_model.predict(X_test)

In [ ]:
mean_absolute_error(y_pred, y_test)

Can use Pipelines, FeatureUnions, FunctionTransformers:
- to systematically search basis function spaces
- Get best fits based off series of functions
- Best function selection

**Obviously very powerful**

Combining FeatureUnion with ColumnTransformer:
- Apply function finding/feature engineering of different kinds to different columns
- Input transformed features into various types of predictive models (with regularization/hyperparameter tuning)

**Now can start to build very powerful/flexible models**

Kind of like building legos. 
- Maybe you want some extra pieces
- Model/Transformer extensions
- Treat models like transformers in FeatureUnion (parallel modeling)
- Get an expansion pack!


May be worth exploring **sklego**:
<center><img src = "Images/sklego.png" width = 300 /></center>

<center>https://scikit-lego.netlify.app/</center>

#### Optimizing tuning strategies

As models/pipelines get complex:
- Lots of hyperparameters to play around with
- GridSearch not the best **initial** strategy:
    - Takes forever
    - Want to sample space more effectively.

Randomized Search CV

<img src = "Images/grid_vs_randomcv.png" />

<center>Often smaller number of random hyperparamter samples better than grid search.</center>

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

Basically same syntax as GridSearchCV:
- param_distributions: 
    - can take dict of hyperparameters with value list: uniformly samples
    - can take dict of hyperparameters with specified distributions.
- n_iter is # of samples to take.

In [ ]:
# will uniformly random sample this grid.
params

In [ ]:
random_cv = RandomizedSearchCV(estimator = regress_pipe, param_distributions = params, n_iter = 10, cv = 6)
random_cv.fit(X_train, y_train);

In [ ]:
randomcv_results = pd.DataFrame(random_cv.cv_results_)
randomcv_results.head()

In [ ]:
best_random = random_cv.best_estimator_
best_random

In [ ]:
random_cv.best_score_

Then after promising RandomSearchCV():
- GridSearchCV (possibly):
    - select best hyperparameter from random search.
    - fine step grid search tuning around this point.

This is just the beginning of hyperperparameter tuning strategies:
- Principled Bayesian approaches exist, etc.
- That being said, RandomSearchCV used well is **very** effective